In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.4 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import Sastrawi
from wordcloud import WordCloud

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-product-at-e-commerce-tokopedia/data_real.csv


In [3]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "data_real.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "faceit24/sentiment-analysis-product-at-e-commerce-tokopedia",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

First 5 records:    Unnamed: 0.1  Unnamed: 0  \
0             0          59   
1             1          70   
2             2          95   
3             3         117   
4             4         153   

                                              Ulasan  Rating     Kategori  \
0                  Pesanan gx sesuai dgn yg d gambar       2  pertukangan   
1  Pengiriman dan respon super cepat. Tpi barang ...       2  pertukangan   
2             barang yg dikirim tidak sesuai pesanan       1  pertukangan   
3  Php, bilang isi ada diseskripsi pas dipesen be...       1  pertukangan   
4    barang jelek copot&#34; terus tmpat taro isinya       2  pertukangan   

                                         Nama Produk  Id Produk Terjual  \
0                 Isi Staples Tembak 8 mm best guard  133507638     545   
1                 Isi Staples Tembak 8 mm best guard  133507638     545   
2                 Isi Staples Tembak 8 mm best guard  133507638     545   
3  steples tembak / staples temba

/tmp/ipykernel_13/1714347155.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


In [4]:
df_copy = df.copy()

## B. Labeling

In [5]:
# Pengkondisian terhadap rating data online shop

label = []
for index, row in df_copy.iterrows():
    if row['Rating'] == 1 or row["Rating"] == 2:
        label.append(0)
    else :
        label.append(1)

df_copy["label"] = label

In [6]:
df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0


In [7]:
def casefolding(Review):
    Review = Review.lower()
    return Review

df_copy['Ulasan'] = df_copy['Ulasan'].apply(casefolding)
df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,pengiriman dan respon super cepat. tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0


In [8]:
norm = {" dgn ":" dengan ", " gue ":" saya ", "bgmn ":" bagaimana ", " tdk":" tidak ", " blum":" belum ", "mantaaaaaaaaaaaappp":" bagus ", "trims":" terima kasih ", "brg":" barang ", "gx":" tidak ", " recommended":" rekomen ", "recomend":"rekomen", "good":" bagus ", " tpi ":" tapi ", " yg ":" yang "}

def normalisasi(str_text):
    for i in norm:
        str_text = str_text.replace(i, norm[i])
    return str_text

df_copy['Ulasan'] = df_copy['Ulasan'].apply(lambda x: normalisasi(x))

df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,pesanan tidak sesuai dengan yang d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,pengiriman dan respon super cepat. tapi barang...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang yang dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0


## C2. StopWords

In [9]:
# Menghapus kata yang memiliki informasi yang rendah. ex: yang, dan, di, dari, dll

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

more_stop_words = []

stop_words = StopWordRemoverFactory().get_stop_words()
new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
    str_text = stop_words_remover_new.remove(str_text)
    return str_text

df_copy['Ulasan'] = df_copy['Ulasan'].apply(lambda x: stopword(x))
df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,pesanan sesuai yang d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,pengiriman respon super cepat. barang sesuai g...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang dikirim sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"php, bilang isi diseskripsi pas dipesen berbed...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0


## C3. Tokenize

In [10]:
# Memisahkan kalimat, paragraf atau dokumen menjadi kata-kata / bagian tertentu, ex: produk ini bagus: ['produk', 'ini', 'bagus']

tokenized = df_copy['Ulasan'].apply(lambda x:x.split())
tokenized

0                      [pesanan, sesuai, yang, d, gambar]
1       [pengiriman, respon, super, cepat., barang, se...
2                      [barang, dikirim, sesuai, pesanan]
3       [php,, bilang, isi, diseskripsi, pas, dipesen,...
4       [barang, jelek, copot&#34;, terus, tmpat, taro...
                              ...                        
1920           [bagus, memuaskan., packing, bagus, cepat]
1921               [mantap, pengiriman, cepat, rekomened]
1922                        [pengiriman, cepat., lumayan]
1923              [belom, dicoba, semoga, oke, barangnya]
1924    [barang, cukup, bagus,, respon, bagus., gak, m...
Name: Ulasan, Length: 1925, dtype: object